In [4]:
import pandas as pd
import re
import string
import sys
sys.path.append('/home/anya/aeneas')

from aeneas.executetask import ExecuteTask
from aeneas.task import Task
from pydub import AudioSegment
from pydub.playback import play
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

In [5]:
def remove_trash(phrase):
    for char in '[]{}()\"\"':
        phrase = phrase.replace(char, '')

    return phrase

# Создание файлов входных данных для aeneas

In [1]:
SEGMENT_ID = 0
RANGE_SEGMENT = range(0, 213)
DELAY = 1.

In [129]:
for input_number in RANGE_SEGMENT:
    prefix = '{"results": '
    try:
        with open('/home/anya/Semantic-Atlas/data/aligned_segments/segment{}/gr_phrases/{}.txt'.format(SEGMENT_ID,
                                                                                                       input_number)) as f:
            text = f.read()
            text = text.rstrip()
            text = text[len(prefix):-1].split('}, {')

        with open('/home/anya/Semantic-Atlas/data/aligned_segments/segment{}/aninp_phrases/{}.txt'.format(SEGMENT_ID,
                                                                                                          input_number),'w') as f_out:
            for output in text:
                for matching in re.finditer('"transcript": ', output):
                    phrase = output[matching.end():]
                    phrase = remove_trash(phrase).split()
                    for word in phrase:
                        f_out.write(word + '\n')
    except Exception as e:
        print input_number, e
        

# Выравнивание распознавания с помощью aeneas

In [130]:
for input_number in RANGE_SEGMENT:
    input_file = '/home/anya/Semantic-Atlas/data/aligned_segments/segment{}/aninp_phrases/{}.txt'.format(SEGMENT_ID,
                                                                                                         input_number)
    output_file = '/home/anya/Semantic-Atlas/data/aligned_segments/segment{}/anout_phrases/{}.txt'.format(SEGMENT_ID,
                                                                                                          input_number)
    audio_path = '/home/anya/Semantic-Atlas/data/aligned_segments/segment{}/audio_phrases/{}.wav'.format(SEGMENT_ID,
                                                                                                         input_number)
    config_string = u"task_language=deu|is_text_type=plain|os_task_file_format=txt|"

    try:
        task = Task(config_string=config_string)
        task.audio_file_path_absolute = audio_path
        task.text_file_path_absolute = input_file
        task.sync_map_file_path_absolute = output_file

        ExecuteTask(task).execute()
        task.output_sync_map_file()

    except Exception as e:
        with open(output_file, 'w') as f:
            pass
        print input_number, e

1600 The task text file seems to have no text fragments
1605 The task text file seems to have no text fragments
1606 The task text file seems to have no text fragments
1622 The task text file seems to have no text fragments
1625 The task text file seems to have no text fragments
1630 The task text file seems to have no text fragments
1639 The task text file seems to have no text fragments
1641 The task text file seems to have no text fragments
1652 The task text file seems to have no text fragments
1653 The task text file seems to have no text fragments
1656 The task text file seems to have no text fragments
1662 The task text file seems to have no text fragments
1672 The task text file seems to have no text fragments
1675 The task text file seems to have no text fragments
1702 The task text file seems to have no text fragments


In [131]:
segment = pd.read_csv('/home/anya/Semantic-Atlas/data/aligned_segments/segment{}/segment{}_phrases.csv'.format(SEGMENT_ID,
                                                                                                              SEGMENT_ID))
segment.index = segment[u'Unnamed: 0']

In [132]:
segment.head()

,Unnamed: 0,start,end,text
Unnamed: 0,,,,
1591,1591,6417.60,6421.900,ss auf einem spielplatz in einem park schaukel...
1592,1592,6424.35,6428.628,forrest ich bin krank
1593,1593,6431.07,6434.426,was hast du einen husten wegen einer erkaeltung
1594,1594,6434.51,6439.584,ich habe einen virus die aerzte wissen nicht w...
1595,1595,6439.67,6442.707,und es gibt nichts was sie dagegen tun koennen


In [133]:
start_segment = segment.start[min(segment.index)] - DELAY
with open('/home/anya/Semantic-Atlas/data/aligned_segments/segment{}/segment{}_words.txt'.format(SEGMENT_ID,\
                                                                                                 SEGMENT_ID), 'w') as f_out:
    for i in RANGE_SEGMENT:
        try:
            audio_path = '/home/anya/Semantic-Atlas/data/aligned_segments/segment{}/audio_phrases/{}.wav'.format(SEGMENT_ID,i)

            with open('/home/anya/Semantic-Atlas/data/aligned_segments/segment{}/anout_phrases/{}.txt'.format(SEGMENT_ID, i)) as f:
                for str_out in f.readlines():
                    id_w, start_w, end_w, word = str_out.rstrip().split(' ')
                    f_out.write('{}\t{}\t{}\n'.format(float(start_w) + segment.ix[i].start - start_segment, 
                                                      float(end_w) + segment.ix[i].start - start_segment, 
                                                      word[1:-1]))

            #audio = AudioSegment.from_file(audio_path)
            #start += len(audio) / 1000.
        except Exception as e:
            print i, e

# Выравнивание сбутитров с помощью aeneas

In [124]:
def write_aeneas_input(segment_id):
    folder_path = "/home/anya/Semantic-Atlas/data/aligned_segments/segment{}/".format(segment_id)
    segment = pd.read_csv(folder_path + "segment{}_phrases.csv".format(segment_id))
    segment.index = segment['Unnamed: 0']
    
    for ind, row in segment.iterrows():
        with open(folder_path + "aninp_srt/{}.txt".format(ind), 'w') as f:
            for word in row.text.split():
                if word in ['ss', 's', '']:
                    continue
                else:
                    f.write(word + '\n')

In [125]:
def align_aeneas(segment_id, segment_range):
    folder_path = "/home/anya/Semantic-Atlas/data/aligned_segments/segment{}/".format(segment_id)
    config_string = u"task_language=deu|is_text_type=plain|os_task_file_format=txt|"
    
    for input_number in segment_range:
        try:
            input_file = folder_path + 'aninp_srt/{}.txt'.format(input_number)
            output_file = folder_path + 'anout_srt/{}.txt'.format(input_number)
            audio_path = folder_path + 'audio_phrases/{}.wav'.format(input_number)

            task = Task(config_string=config_string)
            task.audio_file_path_absolute = audio_path
            task.text_file_path_absolute = input_file
            task.sync_map_file_path_absolute = output_file

            ExecuteTask(task).execute()
            task.output_sync_map_file()
        except Exception as e:
            print input_number, e

In [114]:
def merge_phrases(segment_id, segment_range, delay=0.):
    folder_path = "/home/anya/Semantic-Atlas/data/aligned_segments/segment{}/".format(segment_id)
    segment = pd.read_csv(folder_path + "segment{}_phrases.csv".format(segment_id))
    segment.index = segment['Unnamed: 0']

    start_segment = segment.start[min(segment.index)] - delay
    with open(folder_path + '/segment{}_words_srt.txt'.format(segment_id), 'w') as f_out:
        for i in segment_range:
            try:
                audio_path = folder_path + 'audio_phrases/{}.wav'.format(i)

                with open(folder_path + 'anout_srt/{}.txt'.format(i)) as f:
                    for str_out in f.readlines():
                        id_w, start_w, end_w, word = str_out.rstrip().split(' ')
                        f_out.write('{}\t{}\t{}\n'.format(float(start_w) + segment.ix[i].start - start_segment, 
                                                          float(end_w) + segment.ix[i].start - start_segment, 
                                                          word[1:-1]))
            except Exception as e:
                print i, e

In [115]:
def process_segment(segment_id, delay=0.):
    folder_path = "/home/anya/Semantic-Atlas/data/aligned_segments/segment{}/".format(segment_id)
    segment = pd.read_csv(folder_path + "segment{}_phrases.csv".format(segment_id))
    segment.index = segment['Unnamed: 0']
    start_ind = min(segment.index)
    end_ind = max(segment.index)
    
    write_aeneas_input(segment_id)
    align_aeneas(segment_id, range(start_ind, end_ind + 1))
    merge_phrases(segment_id, range(start_ind, end_ind + 1), delay=delay)

In [ ]:
process_segment(0, delay=1.)

In [49]:
process_segment(1)

439 File '/home/anya/Semantic-Atlas/data/aligned_segments/segment1/audio_phrases/439.wav' cannot be read
439 [Errno 2] No such file or directory: '/home/anya/Semantic-Atlas/data/aligned_segments/segment1/anout_srt/439.txt'


In [94]:
process_segment(2, delay=-0.5)

In [105]:
process_segment(3, delay=-0.7)

In [116]:
process_segment(4, delay=2.2)

In [53]:
process_segment(5)

1310 Audio file format not supported by ffprobe
1310 [Errno 2] No such file or directory: '/home/anya/Semantic-Atlas/data/aligned_segments/segment5/anout_srt/1310.txt'


In [117]:
process_segment(6, delay=7.)

In [126]:
process_segment(7, delay=5.)

In [91]:
previous_phrase = download_phrase(2)

for i in range(3,10):
    current_phrase = download_phrase(i)
    alignment = pairwise2.align.globalxx(previous_phrase, current_phrase, one_alignment_only=True)[0]
    print '+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++'
    print "str1: ", previous_phrase
    print 'str2: ', current_phrase
    print
    print format_alignment(*alignment)
    print 'result: ', find_starting_offset_of_phrases(previous_phrase, current_phrase)
    print
    previous_phrase = current_phrase

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
str1:  Nation auf einem schroffen Berg mit schneebedeckter Flanke fliegt eine Reihe Sterne
str2:  auf einen schroffen Berg mit Schnee bedeckt der Flanke fliegt eine Reihe Sterne zu sie bilden an den

Nation auf einem- schroffen Berg mit s-chnee-bedeckt--er Flanke fliegt eine Reihe Sterne---------------------
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
-------auf eine-n schroffen Berg mit -Schnee bedeckt der Flanke fliegt eine Reihe Sterne zu sie bilden an den
  Score=74

result:  Nation

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
str1:  auf einen schroffen Berg mit Schnee bedeckt der Flanke fliegt eine Reihe Sterne zu sie bilden an den
str2:  hat mit schneebedeckter Flanke fliegt eine Reihe Sterne zu sie bilden an den Kranz und den Kids

auf einen schroffen Berg-

In [92]:
alignment = pairwise2.align.globalxx('Nation', '-at-i-on', one_alignment_only=True)[0]
print format_alignment(*alignment)

N-at-i-on
|||||||||
--at-i-on
  Score=5

